In [ ]:
def create_lf_all(param_info_file, redshift, band_type, band_or_colour_pairs, lf_data_dir, plots_dir_1P, uvlf_limits):
    """Create grid plot of all parameters for a given redshift for LFs."""
    print("\n=== DETAILED DEBUG INFO ===")
    print(f"Parameters received:")
    print(f"param_info_file: {param_info_file}")
    print(f"redshift: {redshift}")
    print(f"band_type: {band_type}")
    print(f"band_or_colour_pairs: {band_or_colour_pairs}")
    print(f"uvlf_limits: {uvlf_limits}")
    
    if not os.path.exists(param_info_file):
        print(f"ERROR: param_info_file does not exist: {param_info_file}")
        return
        
    filter_system = get_safe_name(band_or_colour_pairs, filter_system_only=True)
    band_name = get_safe_name(band_or_colour_pairs)
    data_dir = os.path.join(lf_data_dir[band_type][filter_system], get_safe_name(redshift['label']))
    
    try:
        params_df = pd.read_csv(param_info_file)
        print(f"\nSuccessfully loaded parameter info file with {len(params_df)} parameters")
    except Exception as e:
        print(f"ERROR: Failed to read param_info_file: {str(e)}")
        return
    
    # Changed grid layout for 6 parameters (2x3)
    nrows, ncols = 2, 3
    fig, axes = plt.subplots(nrows, ncols, figsize=(15, 10))
    axes = axes.flatten()
    
    variations = ['n2', 'n1', '1', '2']
    colours = ['blue', 'green', 'red', 'purple']
    plots_made = 0
    
    # Only loop through 6 parameters instead of 28
    for param_num in range(1, 7):
        ax = axes[param_num-1]
        param_info = params_df.iloc[param_num-1]
        param_values = calculate_parameter_values(param_info)
        
        panel_plots = 0
        print(f"\nProcessing parameter {param_num}: {param_info['ParamName']}")
        
        for var, colour in zip(variations, colours):
            filename = os.path.join(data_dir, 
                                  f"UVLF_1P_p{param_num}_{var}_{band_name}_{get_safe_name(redshift['label'])}_{band_type}.txt")
            
            if os.path.exists(filename):
                try:
                    data = pd.read_csv(filename, delimiter='\t')
                    ax.plot(data['magnitude'], data['phi'], 
                          color=colour, linewidth=1.5,
                          label=f'{param_values[var]:.3g}')
                    panel_plots += 1
                    plots_made += 1
                except Exception as e:
                    print(f"ERROR reading/plotting {filename}: {str(e)}")
        
        if panel_plots > 0:
            ax.set_xlim(*uvlf_limits)
            ax.set_ylim(-6, -2)
            ax.grid(True, which='both', linestyle='--', alpha=0.3)
            ax.set_title(f'p{param_num}: {param_info["ParamName"]}', fontsize=10)  # Increased font size
            ax.tick_params(axis='both', which='major', labelsize=8)  # Increased tick label size
            
            if param_num == 1:
                ax.legend(fontsize=8, title='Values', title_fontsize=8)
    
    # Add overall title
    fig.suptitle(f'{band_name} UVLF Parameter Variations (z = {redshift["redshift"]})', 
                 fontsize=14, y=1.02)
    
    plt.tight_layout()
    
    # Save plot
    plot_dir = os.path.join(plots_dir_1P, 'LFs', band_type, band_name, get_safe_name(redshift['label']))
    os.makedirs(plot_dir, exist_ok=True)
    
    output_file = os.path.join(plot_dir, 
                              f"All_UVLF_{band_name}_{get_safe_name(redshift['label'])}_{band_type}.pdf")
    print(f"\nSaving plot to: {output_file}")
    plt.savefig(output_file, bbox_inches='tight', dpi=300)
    plt.close()